## ENVIRONMENT VARIABLES

In [8]:
QUY_ES_CORPUS_DIR = "/notebooks/master-thesis/corpora/americasnlp2021/data/quechua-spanish"

In [9]:
tokenizer_path = "/storage/master-thesis/models/quy-es/tokenizers/quy-es/"

In [10]:
%env TOKENIZER_PATH = /storage/master-thesis/models/quy-es/tokenizers/quy-es

env: TOKENIZER_PATH=/storage/master-thesis/models/quy-es/tokenizers/quy-es


In [11]:

tokenized_path_quy_es = "/storage/master-thesis/models/quy-es/tokenizers/quy-es/quy-es/"

In [12]:

%env TOKENIZED_PATH_QUY_ES = /storage/master-thesis/models/quy-es/tokenizers/quy-es/quy-es

env: TOKENIZED_PATH_QUY_ES=/storage/master-thesis/models/quy-es/tokenizers/quy-es/quy-es


In [13]:
%env BIN_DIR = /storage/master-thesis/models/quy-es/tokenizers/quy-es/bin

env: BIN_DIR=/storage/master-thesis/models/quy-es/tokenizers/quy-es/bin


In [14]:
%env MODEL_DIR = /storage/master-thesis/models/quy-es

env: MODEL_DIR=/storage/master-thesis/models/quy-es


In [15]:

! mkdir -p $TOKENIZED_PATH_QUY_ES
! mkdir -p $BIN_DIR

In [10]:
! locale

LANG=
LANGUAGE=
LC_CTYPE="POSIX"
LC_NUMERIC="POSIX"
LC_TIME="POSIX"
LC_COLLATE="POSIX"
LC_MONETARY="POSIX"
LC_MESSAGES="POSIX"
LC_PAPER="POSIX"
LC_NAME="POSIX"
LC_ADDRESS="POSIX"
LC_TELEPHONE="POSIX"
LC_MEASUREMENT="POSIX"
LC_IDENTIFICATION="POSIX"
LC_ALL=


In [11]:
! update-locale LANG=en_US.UTF-8 LANGUAGE=en.UTF-8

%env LANG=en_US.UTF-8
%env LC_CTYPE=en_US.UTF-8
%env LC_ALL=en_US.UTF-8

/bin/sh: 1: update-locale: not found
env: LANG=en_US.UTF-8
env: LC_CTYPE=en_US.UTF-8
env: LC_ALL=en_US.UTF-8


## Libraries

In [14]:
pip install tokenizers==0.10.3

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip uninstall torch torchvision torchaudio -y

Found existing installation: torch 1.10.2+cu113
Uninstalling torch-1.10.2+cu113:
  Successfully uninstalled torch-1.10.2+cu113
Found existing installation: torchvision 0.11.3+cu113
Uninstalling torchvision-0.11.3+cu113:
  Successfully uninstalled torchvision-0.11.3+cu113
Found existing installation: torchaudio 0.10.2+cu113
Uninstalling torchaudio-0.10.2+cu113:
  Successfully uninstalled torchaudio-0.10.2+cu113
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
  Using cached https://download.pytorch.org/whl/cu113/torch-1.10.2%2Bcu113-cp36-cp36m-linux_x86_64.whl (1821.5 MB)
  Using cached https://download.pytorch.org/whl/cu113/torchvision-0.11.3%2Bcu113-cp36-cp36m-linux_x86_64.whl (24.6 MB)
  Using cached https://download.pytorch.org/whl/cu113/torchaudio-0.10.2%2Bcu113-cp36-cp36m-linux_x86_64.whl (2.9 MB)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [17]:
pip uninstall fairseq -y

Found existing installation: fairseq 0.12.2
Uninstalling fairseq-0.12.2:
  Successfully uninstalled fairseq-0.12.2
Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install fairseq # --no-deps

  Using cached fairseq-0.12.2-cp36-cp36m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.0 MB)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [19]:
! pip uninstall apex -y

Found existing installation: apex 0.9.10.dev0
Uninstalling apex-0.9.10.dev0:
  Successfully uninstalled apex-0.9.10.dev0


In [20]:
rm -r apex

rm: cannot remove 'apex': No such file or directory


In [ ]:
#! git clone https://github.com/NVIDIA/apex

! pip install -v --disable-pip-version-check --no-cache-dir apex

In [22]:
pip install tensorboardX

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Converting tatoeba format to standard format

In [ ]:
import csv

with open(EN_ES_CORPUS_DIR + "original/valid.txt", encoding='utf8') as f:
    es = open(EN_ES_CORPUS_DIR + "dev.es", 'w', encoding='utf8')
    en = open(EN_ES_CORPUS_DIR + "dev.en", 'w', encoding='utf8')
    for line in csv.reader(f, delimiter="\t"):
        es.write(line[3] + "\n")
        en.write(line[2] + "\n")
    en.close()
    es.close()
    
with open(EN_ES_CORPUS_DIR + "original/dev.txt", encoding='utf8') as f:
    es = open(EN_ES_CORPUS_DIR + "train.es", 'w', encoding='utf8')
    en = open(EN_ES_CORPUS_DIR + "train.en", 'w', encoding='utf8')
    for line in csv.reader(f, delimiter="\t"):
        if "\n" in line[2] or "\n" in line[3]:
            continue
        if len(line) != 4:
            continue
            
        es.write(line[3] + "\n")
        en.write(line[2] + "\n")
        
    en.close()
    es.close()
        

### Tokenizing data

In [16]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece

es_quy_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

In [17]:
from tokenizers.trainers import WordPieceTrainer

es_quy_trainer = WordPieceTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"], show_progress=True)

In [18]:
from tokenizers.pre_tokenizers import Whitespace

es_quy_tokenizer.pre_tokenizer = Whitespace()

quy_files = [f"/notebooks/master-thesis/corpora/americasnlp2021/data/quechua-spanish/{split}.quy" for split in ["dev", "train"]]
es2_files = [f"/notebooks/master-thesis/corpora/americasnlp2021/data/quechua-spanish/{split}.es" for split in ["dev", "train"]]


files = es2_files + quy_files
#print(files)
es_quy_tokenizer.train(files= files, trainer=es_quy_trainer)

In [19]:
# ! cat /notebooks/master-thesis/corpora/tatoeba-challenge/en-es/dev.en

In [20]:
from pathlib import Path



def tokenize_files(tokenizer, files, extension, output_path):
    for file in files:
        print(f"Reading file {file}")
        with open(file, encoding='utf8') as f:
          lines = f.readlines()
          tokenized_lines = tokenizer.encode_batch(lines)
          tokenized_name = Path(file).stem
          tokenized_name = output_path + tokenized_name + "." + extension
          print(tokenized_name)
          with open(tokenized_name, 'w', encoding='utf8') as wf:

            wf.writelines([" ".join(t.tokens) + "\n" for t in tokenized_lines])

In [21]:
es_quy_tokenizer.save(tokenizer_path + "es-quy-tokenizer.json")

In [22]:
print(es_quy_tokenizer)

In [23]:
tok = es_quy_tokenizer.encode("we hold this truth to be self evidently that everyone is created equal?")
print(tok.tokens)


['w', '##e', 'hol', '##d', 't', '##hi', '##s', 'tr', '##uth', 'to', 'be', 'sel', '##f', 'evid', '##en', '##t', '##ly', 't', '##ha', '##t', 'ev', '##er', '##y', '##one', 'is', 'crea', '##ted', 'e', '##qu', '##al', '?']


In [25]:


tokenize_files(es_quy_tokenizer, quy_files, "quy", tokenized_path_quy_es)
tokenize_files(es_quy_tokenizer, es2_files, "es", tokenized_path_quy_es)

Reading file /notebooks/master-thesis/corpora/americasnlp2021/data/quechua-spanish/dev.quy
/storage/master-thesis/models/quy-es/tokenizers/quy-es/quy-es/dev.quy
Reading file /notebooks/master-thesis/corpora/americasnlp2021/data/quechua-spanish/train.quy
/storage/master-thesis/models/quy-es/tokenizers/quy-es/quy-es/train.quy
Reading file /notebooks/master-thesis/corpora/americasnlp2021/data/quechua-spanish/dev.es
/storage/master-thesis/models/quy-es/tokenizers/quy-es/quy-es/dev.es
Reading file /notebooks/master-thesis/corpora/americasnlp2021/data/quechua-spanish/train.es
/storage/master-thesis/models/quy-es/tokenizers/quy-es/quy-es/train.es


## Binarizing data

In [26]:
### Removing previous dict files
! rm $BIN_DIR/dict.quy.txt
! rm $BIN_DIR/dict.es.txt

rm: cannot remove '/storage/master-thesis/models/quy-es/tokenizers/quy-es/bin/dict.quy.txt': No such file or directory
rm: cannot remove '/storage/master-thesis/models/quy-es/tokenizers/quy-es/bin/dict.es.txt': No such file or directory


In [23]:
## Concatenating all training data
#! cat $TOKENIZED_PATH_QUY_ES/train.quy $TOKENIZED_PATH_QUY_ES/train.es $TOKENIZED_PATH_AYM_ES/train.aym $TOKENIZED_PATH_AYM_ES/train.es > $BIN_DIR/train.all

In [36]:
ls

 Untitled.ipynb
 bar.txt
 checkpoints/
 den.pt
 docker/
 docs/
 examples/
 master-thesis/
 model_cards/
 norm.pt
 notebooks/
 num.pt
'old scripts'/
 pos.pt
 s.pt
 scripts/
 sj.pt
 src/
 templates/
'test 10nal-es + es-en model.ipynb'
'test capsulenet quy-es + es-en model.ipynb'
'test it-nl-ro-en model.ipynb'
'test nl-it-en + de-en model.ipynb'
'test quy-es + aym-es,en + es-en model.ipynb'
'test quy-es + es-en model.ipynb'
'test quy-es -> es-en model.ipynb'
'test residual drop quy-es + es-en model.ipynb'
 tests/
'train 10nal-es + es-en model.ipynb'
'train aym-es,es-en.ipynb'
'train capsulenet nl-it-en + de-en model.ipynb'
'train capsulenet quy-es + es-en model.ipynb'
'train es-en model.ipynb'
'train it-nl-ro-en model.ipynb'
'train nah-es,es-en.ipynb'
'train nl-en,it-en,ro-en,en-de.ipynb'
'train quy-es + aym-es,en + es-en model.ipynb'
'train quy-es,aym-es,es-en.ipynb'
'train quy-es,es-en.ipynb'
'train residual drop quy-es + es-en model.ipynb'
 utils/
 vj.pt


In [24]:
#! fairseq-preprocess --source-lang all \
#    --trainpref $BIN_DIR/train \
#    --destdir $BIN_DIR \
#    --workers 20 \
#    --only-source

2022-09-19 21:50:17 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/storage/master-thesis/models/quy-es++aym-es/tokenizers/quy-aym-es/bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=True, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False,

In [25]:
! echo $BIN_DIR

/storage/master-thesis/models/quy-es++aym-es/tokenizers/quy-aym-es/bin


In [27]:
! fairseq-preprocess --source-lang quy --target-lang es \
    --trainpref $TOKENIZED_PATH_QUY_ES/train --validpref $TOKENIZED_PATH_QUY_ES/dev \
    --destdir $BIN_DIR \
    --workers 20

2022-09-20 21:23:49 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/storage/master-thesis/models/quy-es/tokenizers/quy-es/bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='b

## Training model

In [28]:
! fairseq-train $BIN_DIR \
    --arch=transformer \
    --task translation_multi_simple_epoch --lang-pairs quy-es \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --restore-file $MODEL_DIR/checkpoint_last.pt \
    --save-dir $MODEL_DIR/ \
    --keep-last-epochs 2 \
    --reset-optimizer \
    --encoder-langtok "src" \
    --decoder-langtok \
    --fp16 \
    --max-epoch 200 \
    --patience 10


2022-09-20 21:24:16 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': No

In [34]:
! echo $MODEL_DIR

/storage/master-thesis/models/quy-es+es-en


In [36]:
cd /storage/master-thesis/models/aym-es+es-en

/storage/master-thesis/models/quy-es+es-en


In [29]:
! rm dict.en.txt
! rm dict.es.txt
! rm dict.aym.txt

In [84]:
! du -shc /notebooks/*

2.5K	/notebooks/CITATION.cff
5.5K	/notebooks/CODE_OF_CONDUCT.md
15K	/notebooks/CONTRIBUTING.md
19K	/notebooks/ISSUES.md
12K	/notebooks/LICENSE
512	/notebooks/MANIFEST.in
3.5K	/notebooks/Makefile
41K	/notebooks/README.md
41K	/notebooks/README_zh-hans.md
42K	/notebooks/README_zh-hant.md
12K	/notebooks/docker
4.6M	/notebooks/docs
5.0M	/notebooks/examples
8.5K	/notebooks/hubconf.py
7.8G	/notebooks/master-thesis
1.5K	/notebooks/model_cards
9.5K	/notebooks/notebooks
512	/notebooks/pyproject.toml
64K	/notebooks/scripts
1.0K	/notebooks/setup.cfg
13K	/notebooks/setup.py
13M	/notebooks/src
731K	/notebooks/templates
4.5K	/notebooks/test quy-es -> es-en model.ipynb
6.8M	/notebooks/tests
147K	/notebooks/train es-en model.ipynb
158K	/notebooks/train quy-es + es-en model.ipynb
160K	/notebooks/utils
3.5K	/notebooks/valohai.yaml
7.9G	total


In [85]:
! du -shc /notebooks/master-thesis/*

512	/notebooks/master-thesis/Untitled.ipynb
362M	/notebooks/master-thesis/corpora
7.5G	/notebooks/master-thesis/models
7.8G	total


In [86]:
! du -shc /notebooks/master-thesis/models/*

1.6G	/notebooks/master-thesis/models/es-en
802M	/notebooks/master-thesis/models/quy-es
5.2G	/notebooks/master-thesis/models/quy-es+es-en
7.5G	total


In [78]:
! du -shc /*

19M	/apex
5.0M	/bin
4.0K	/boot
24K	/content
^C


In [89]:
ls

 CITATION.cff         docker/          setup.py
 CODE_OF_CONDUCT.md   docs/            src/
 CONTRIBUTING.md      examples/        templates/
 ISSUES.md            hubconf.py      'test quy-es -> es-en model.ipynb'
 LICENSE              master-thesis/   tests/
 MANIFEST.in          model_cards/    'train es-en model.ipynb'
 Makefile             notebooks/      'train quy-es + es-en model.ipynb'
 README.md            pyproject.toml   utils/
 README_zh-hans.md    scripts/         valohai.yaml
 README_zh-hant.md    setup.cfg


/storage/master-thesis
